<a href="https://colab.research.google.com/github/Tiya4425/-datasciencecoursera/blob/main/topic_modeling_and_sentence_transformers_using_phising_email_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset is from : https://github.com/IBM/nlc-email-phishing

# Dataset Acquisition

In [ ]:
!wget https://raw.githubusercontent.com/IBM/nlc-email-phishing/master/data/Email-trainingdata-20k.csv

--2022-12-25 13:59:14--  https://raw.githubusercontent.com/IBM/nlc-email-phishing/master/data/Email-trainingdata-20k.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8626842 (8.2M) [text/plain]
Saving to: ‘Email-trainingdata-20k.csv’

Email-trainingdata- 100%[===================>]   8.23M  --.-KB/s    in 0.09s   

2022-12-25 13:59:14 (94.3 MB/s) - ‘Email-trainingdata-20k.csv’ saved [8626842/8626842]



In [ ]:
import pandas as pd
df= pd.read_csv('/content/Email-trainingdata-20k.csv', names=['text','phishing', 'label'],header=None)

In [ ]:
df=df.iloc[1:, :]

In [ ]:
df.head()

,text,phishing,label
1,"Subject: what up , , your cam babe what are yo...",phishing,spam
2,Subject: want to make more money ? order confi...,phishing,spam
3,Subject: food for thoughts [ join now - take a...,phishing,spam
4,Subject: your pharmacy ta would you want cheap...,phishing,spam
5,Subject: bigger breast just from a pill image ...,phishing,spam


In [ ]:
df['text'] = df['text'].str.replace('Subject', '')

In [ ]:
df['text'] = df['text'].str.replace(':', '')

In [ ]:
df.head()

,text,phishing,label
1,"what up , , your cam babe what are you lookin...",phishing,spam
2,want to make more money ? order confirmation ...,phishing,spam
3,food for thoughts [ join now - take a free to...,phishing,spam
4,your pharmacy ta would you want cheap perscri...,phishing,spam
5,bigger breast just from a pill image is loadi...,phishing,spam


In [ ]:
df.phishing.value_counts()

phishing    17830
Name: phishing, dtype: int64

In [ ]:
df.label.value_counts()

spam    9494
ham     8336
Name: label, dtype: int64

In [ ]:
df.columns

Index(['text', 'phishing', 'label'], dtype='object')

# clean text

In [ ]:
!pip install emoji 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 240 kB 5.7 MB/s 
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=3d263bb84e907fc0e8007daa3a7a575b2183a22d270a40d29e7516602496bdc9
  Stored in directory: /root/.cache/pip/wheels/86/62/9e/a6b27a681abcde69970dbc0326ff51955f3beac72f15696984
Successfully built emoji


In [ ]:
'''import re
import emoji
import nltk
nltk.download('words')
words = set(nltk.corpus.words.words())
def cleaner(tweet):
    tweet = re.sub("@[A-Za-z0-9]+","",tweet) #Remove @ sign
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) #Remove http links
    tweet = " ".join(tweet.split())
    tweet = ''.join(c for c in tweet if c not in emoji.distinct_emoji_list(tweet)) #Remove Emojis
    tweet = tweet.replace("#", "").replace("_", " ") #Remove hashtag sign but keep the text
    tweet = " ".join(w for w in nltk.wordpunct_tokenize(tweet) \
         if w.lower() in words or not w.isalpha())
    return tweet
df['cleaned_text'] = df['text'].map(lambda x: cleaner(x))'''


'import re\nimport emoji\nimport nltk\nnltk.download(\'words\')\nwords = set(nltk.corpus.words.words())\ndef cleaner(tweet):\n    tweet = re.sub("@[A-Za-z0-9]+","",tweet) #Remove @ sign\n    tweet = re.sub(r"(?:\\@|http?\\://|https?\\://|www)\\S+", "", tweet) #Remove http links\n    tweet = " ".join(tweet.split())\n    tweet = \'\'.join(c for c in tweet if c not in emoji.distinct_emoji_list(tweet)) #Remove Emojis\n    tweet = tweet.replace("#", "").replace("_", " ") #Remove hashtag sign but keep the text\n    tweet = " ".join(w for w in nltk.wordpunct_tokenize(tweet)          if w.lower() in words or not w.isalpha())\n    return tweet\ndf[\'cleaned_text\'] = df[\'text\'].map(lambda x: cleaner(x))'

In [ ]:
df.head(100)

,text,phishing,label
1,"what up , , your cam babe what are you lookin...",phishing,spam
2,want to make more money ? order confirmation ...,phishing,spam
3,food for thoughts [ join now - take a free to...,phishing,spam
4,your pharmacy ta would you want cheap perscri...,phishing,spam
5,bigger breast just from a pill image is loadi...,phishing,spam
...,...,...,...
96,- = i suppose outstanding proffer for you at ...,phishing,spam
97,"quick , easy vicodin w lij tirb xhzcixu we ar...",phishing,spam
98,rock her world with the little pill ! click h...,phishing,spam
99,"hi one million profiles of hot old age , youn...",phishing,spam


In [ ]:
import string
import re
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

df['cleaned_text'] = df['text'].apply(lambda x: remove_punct(x))
df.head(10)

,text,phishing,label,cleaned_text
1,"what up , , your cam babe what are you lookin...",phishing,spam,what up your cam babe what are you looking ...
2,want to make more money ? order confirmation ...,phishing,spam,want to make more money order confirmation ...
3,food for thoughts [ join now - take a free to...,phishing,spam,food for thoughts join now take a free tour...
4,your pharmacy ta would you want cheap perscri...,phishing,spam,your pharmacy ta would you want cheap perscri...
5,bigger breast just from a pill image is loadi...,phishing,spam,bigger breast just from a pill image is loadi...
6,cant find you on msn . . . but ride it januar...,phishing,spam,cant find you on msn but ride it january a...
7,majestic rx - - - - 8673849000404220545 hi va...,phishing,spam,majestic rx hi varou how quickly does i...
8,quick way to buy soft - ware variety of top m...,phishing,spam,quick way to buy soft ware variety of top ma...
9,"some advice to him mon , 27 dec 2004 11 16 ...",phishing,spam,some advice to him mon dec good day...
10,software at incredibly low prices ( 83 % lowe...,phishing,spam,software at incredibly low prices lower ...


In [ ]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.3 MB/s 
     |████████████████████████████████| 5.8 MB 11.1 MB/s 
     |████████████████████████████████| 1.3 MB 62.8 MB/s 
     |████████████████████████████████| 182 kB 75.9 MB/s 
     |████████████████████████████████| 7.6 MB 51.0 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=fe975455b472fda6ffba3c8fb34d0ab9de07980324b2b9fbb1a6451ce291dc73
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


In [ ]:
df.head()

,text,phishing,label,cleaned_text
1,"what up , , your cam babe what are you lookin...",phishing,spam,what up your cam babe what are you looking ...
2,want to make more money ? order confirmation ...,phishing,spam,want to make more money order confirmation ...
3,food for thoughts [ join now - take a free to...,phishing,spam,food for thoughts join now take a free tour...
4,your pharmacy ta would you want cheap perscri...,phishing,spam,your pharmacy ta would you want cheap perscri...
5,bigger breast just from a pill image is loadi...,phishing,spam,bigger breast just from a pill image is loadi...


In [ ]:
df.shape

(17830, 1)

In [ ]:
df=df[['cleaned_text']]

In [ ]:
from sentence_transformers import SentenceTransformer

from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import pickle
from joblib import Memory

In [ ]:
df['cleaned_text'] = df['cleaned_text'].astype("string")

In [ ]:
df.head()

,cleaned_text
1,what up your cam babe what are you looking ...
2,want to make more money order confirmation ...
3,food for thoughts join now take a free tour...
4,your pharmacy ta would you want cheap perscri...
5,bigger breast just from a pill image is loadi...


In [ ]:
df.set_index('cleaned_text')

""
cleaned_text
what up your cam babe what are you looking for if your looking for a companion for friendship love a date or just good ole fashioned then try our brand new site it was developed and created to help anyone find what they re looking for a quick bio form and you re on the road to satisfaction in every sense of the word no matter what that may be try it out and youll be amazed have a terrific time this evening copy and pa ste the add ress you see on the line below into your browser to come to the site http www meganbang biz bld acc no more plz http www naturalgolden com retract counterattack aitken step preemptive shoehorn scaup electrocardiograph movie honeycomb monster war brandywine pietism byrne catatonia encomia lookup intervenor skeleton turn catfish
want to make more money order confirmation your order should be shipped by january via fedex your federal express tracking number is random word thank you for registering your userid is random word learn to make a fortune with ebay complete turnkey system software videos turorials clck here if you would not like to receive future mailings
food for thoughts join now take a free tour click here to be removed
your pharmacy ta would you want cheap perscriptions http www prlce com
bigger breast just from a pill image is loading cli k here for more info i was then shipped off to a store after about a week a man bought me as they sent me through rollers i fell asleep expecting never to awaken after i was sent through rollers i was sent off to a strange place called a printing press where i was made into a newspaper
...
re your consent needed from the desk of honorable chairman local everything is directly under my control i am offering you this believing that you will be capable of keeping the fund in safe custody and will hand over my share of which i will love to invest in your country please if it is okay by you contact me immediately but the only problem is trust if can i trust you send to me your phone number age status office position yours truly prince udo ogali chairman debt reconciliation panel email princeudol yahoo es s listo contrata el antivirus http www pandasoftware es tienda idpers track
see our page save thousands on m e d s we help you get m e d s for cheap zv slb g then i d know i ought therefore as the elephant was sideways onright
here you go timed aaron offload misplaces shoved withstanding breveting kaplan properties reciprocation degradations nice dislike dreary democrats rhythmically neutrinos dealers metcalf shambles nonnegative assuringly diminutive allies collie burma davidson rationalize horns overturn quantities mires countryman dissect gorton animized phone mobile email antonia seward mesh ad jp


In [ ]:
data=df['cleaned_text'].to_list()

In [ ]:
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 4.0 MB/s 
     |████████████████████████████████| 662 kB 26.6 MB/s 
     |████████████████████████████████| 5.2 MB 59.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 88 kB 8.7 MB/s 
     |████████████████████████████████| 1.1 MB 60.9 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.29-cp38-cp38-linux_x86_64.whl size=2700883 sha256=5f3c6623af39b08de86e03e6c3f3f68ec00c0ed3ef9737bc396e50591f4d4a39
  Stored in directory: /root/.cache/pip/wheels/76/06/48/527e038689c581cc9e519c73840efdc7473805149e55bd7ffd
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=4240d71479da496167c59b23fbd5cff6be881a566add7d4166a6612bfe461eca
  Stored in directory: /root/.cache/pip/wheels/a9/3a/67/06a

In [ ]:
from bertopic import BERTopic

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
topic_model = BERTopic(embedding_model=model)

Batches:   0%|          | 0/558 [00:00<?, ?it/s]

In [ ]:
topics, probs = topic_model.fit_transform(data)

In [ ]:
freq = topic_model.get_topic_info(); freq.head(5)

,Topic,Count,Name
0,-1,7034,-1_the_to_on_of
1,0,538,0_mobile_sa_vicodin_phone
2,1,452,1_schedules_hourahead_hour_start
3,2,240,2_suggested_ect_feedback_employee
4,3,224,3_mobile_phone_farm_govern


In [ ]:
topic_model.get_topic(0)

[('mobile', 0.006002043197316973),
 ('sa', 0.0049665139953999834),
 ('vicodin', 0.00480988622509884),
 ('phone', 0.004581355990529552),
 ('off', 0.004394437267326583),
 ('email', 0.003920285378579044),
 ('removemeplease', 0.003322020477488992),
 ('onl', 0.0032938796037688682),
 ('ering', 0.0032427673086211774),
 ('vi', 0.003100012139180646)]

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_barchart(top_n_topics=10)